<a href="https://colab.research.google.com/github/srikarthickeyanganapathy/Smart_Agriculture_Monitoring_System/blob/main/SAMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_excel("/content/GHISACONUS_2008_001_speclib_updated.xlsx")
df.to_csv("SAMS.csv", index=False)

In [4]:
print("Dataset loaded successfully:")
df.head()

Dataset loaded successfully:


,UniqueID,Country,AEZ,Image,Month,Year,jd,long,lat,Crop,...,X2254,X2264,X2274,X2285,X2295,X2305,X2315,X2325,X2335,X2345
0,1466,USA,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.663419,38.534516,corn,...,8.432855,8.438398,8.585301,8.164735,7.882625,7.380592,7.327904,6.881876,6.616288,6.346634
1,1467,USA,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.671589,38.504744,corn,...,8.682515,8.942595,9.179517,8.744633,8.510454,8.090403,7.894839,7.263033,6.536649,5.663291
2,1469,USA,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.597588,38.614056,corn,...,6.841276,6.276240,6.150597,6.114164,5.934033,5.116303,4.833205,4.817732,4.652525,3.790456
3,1470,USA,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.687293,38.571702,corn,...,6.679967,6.534779,6.430438,6.080465,6.201545,6.291307,6.922908,6.943876,6.461839,5.696439
4,1476,USA,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.625189,38.572225,rice,...,5.256362,5.022666,4.988447,5.220789,5.726055,6.007986,4.820400,3.084297,1.322807,1.087548


In [10]:
# --- 1. Separate Features and Target ---
# Identify the columns for each data type
spectral_columns = [col for col in df.columns if 'X' in col]
iot_columns = ['soil_moisture', 'temperature']
target_column = 'yield_tph'

In [11]:
# Create the feature sets (X) and the target (y)
X_spectral = df[spectral_columns].values
X_iot = df[iot_columns].values
y = df[target_column].values

In [12]:
# --- 2. Normalize the Data ---
# It's crucial to scale numerical data for neural networks
scaler_spectral = StandardScaler()
X_spectral_scaled = scaler_spectral.fit_transform(X_spectral)

In [13]:
scaler_iot = StandardScaler()
X_iot_scaled = scaler_iot.fit_transform(X_iot)

In [14]:
# --- 3. Reshape Data for the 1D-CNN ---
# The CNN needs an extra dimension for the "channels"
X_spectral_reshaped = X_spectral_scaled.reshape(X_spectral_scaled.shape[0], X_spectral_scaled.shape[1], 1)

In [15]:
# --- 4. Split Data into Training and Testing Sets ---
X_spec_train, X_spec_test, X_iot_train, X_iot_test, y_train, y_test = train_test_split(
    X_spectral_reshaped, X_iot_scaled, y, test_size=0.2, random_state=42
)

In [16]:
print(f"Spectral training data shape: {X_spec_train.shape}")
print(f"IoT training data shape: {X_iot_train.shape}")
print(f"Yield training data shape: {y_train.shape}")

Spectral training data shape: (5590, 131, 1)
IoT training data shape: (5590, 2)
Yield training data shape: (5590,)


In [45]:
# --- Define the Spectral Branch (1D-CNN) ---
# Explicitly set the input shape based on the data dimensions (131 features)
spectral_input = Input(shape=(X_spec_train.shape[1], 1), name='spectral_input')
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(spectral_input)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)
spectral_flat = Flatten()(pool2)

In [46]:
# --- Define the IoT Branch (Dense Layers) ---
iot_input = Input(shape=(X_iot_train.shape[1],), name='iot_input')
iot_dense = Dense(8, activation='relu')(iot_input)

In [47]:
# --- Fuse the two branches ---
fused_layer = Concatenate()([spectral_flat, iot_dense])

In [48]:
# --- Add Dense layers for final processing ---
dense1 = Dense(64, activation='relu')(fused_layer)
dense2 = Dense(32, activation='relu')(dense1)

In [49]:
# --- Output Layer for Regression ---
# A single neuron with a linear activation for predicting a continuous value
output = Dense(1, activation='linear', name='output')(dense2)

In [50]:
# --- Create the final model ---
model = Model(inputs=[spectral_input, iot_input], outputs=output)

In [51]:
# --- Compile the model ---
# For regression, we use Mean Squared Error as the loss function
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ spectral_input      │ (None, 131, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 129, 32)   │        128 │ spectral_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 64, 32)    │          0 │ conv1d_10[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 62, 64)    │      6,208 │ max_pooling1d_10… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_11    │ (None, 31, 64)    │          0 │ conv1d_11[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ iot_input           │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 1984)      │          0 │ max_pooling1d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 8)         │         24 │ iot_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 1992)      │          0 │ flatten_5[0][0],  │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │    127,552 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      2,080 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 136,025 (531.35 KB)

 Trainable params: 136,025 (531.35 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
print("Starting model training...")
history = model.fit(
    {'spectral_input': X_spec_train, 'iot_input': X_iot_train},
    y_train,
    epochs=50,  # You can adjust the number of epochs
    batch_size=32,
    validation_split=0.2, # Use part of the training data for validation
    verbose=1
)
print("Training complete.")

Starting model training...
Epoch 1/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 4.8498 - root_mean_squared_error: 2.0321 - val_loss: 0.0561 - val_root_mean_squared_error: 0.2368
Epoch 2/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0365 - root_mean_squared_error: 0.1907 - val_loss: 0.0237 - val_root_mean_squared_error: 0.1539
Epoch 3/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0190 - root_mean_squared_error: 0.1375 - val_loss: 0.0123 - val_root_mean_squared_error: 0.1109
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0106 - root_mean_squared_error: 0.1028 - val_loss: 0.0117 - val_root_mean_squared_error: 0.1081
Epoch 5/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0081 - root_mean_squared_error: 0.0898 - val_loss: 0.0475 - val_root_mean_squared_error: 0.2179
Epoch 6/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0128 - root_mean_squared_error: 0.1109 - val_loss: 0.0041 - val_root_mean_squared_error: 0.0637
Epoch 7/50
140/140 

In [53]:
print("Evaluating model performance on the test set...")
results = model.evaluate(
    {'spectral_input': X_spec_test, 'iot_input': X_iot_test},
    y_test
)

Evaluating model performance on the test set...
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0012 - root_mean_squared_error: 0.0348


In [54]:
print(f"Test Loss: {results[0]}")
print(f"Test Root Mean Squared Error (RMSE): {results[1]}")

Test Loss: 0.0011685462668538094
Test Root Mean Squared Error (RMSE): 0.03418400511145592
